In [5]:
import pandas as pd

# Cargar datos
archivo_entrada = '../vehicles_us.csv'
archivo_salida = '../vehicles_us_limpio.csv'
df = pd.read_csv(archivo_entrada)

# Llenar valores faltantes

# model y paint_color
df['model'].fillna('Otro', inplace=True)
df['paint_color'].fillna('Otro', inplace=True)

# model_year, odometer, cylinder y is_4WD
df['model_year'] = df['model_year'].fillna(0).astype(int)
df['odometer'] = df['odometer'].fillna(0).astype(int)
df['cylinders'] = df['cylinders'].fillna(0).astype(int)
df['is_4wd'] = df['is_4wd'].fillna(3).astype(int)

# is_4wd -> texto
is_4wd_map = {1: 'Sí', 0: 'Sí',3: '-'}
df['is_4wd'] = df['is_4wd'].map(is_4wd_map).fillna('No').astype(str)

# Dividir columna model
df[['Marca', 'Modelo']] = (
    df['model']
    .str.extract(r'^(\w+)\s+(.*)', expand=True)
    .apply(lambda col: col.str.strip().str.title())
)
df.drop(columns='model', inplace=True)

# Renombrar columnas
df.rename(columns={
    'price': 'Precio',
    'model_year': 'Año del modelo',
    'condition': 'Condición',
    'cylinders': 'Cilindrada',
    'fuel': 'Combustible',
    'odometer': 'Kilometraje',
    'transmission': 'Transmisión',
    'type': 'Tipo',
    'paint_color': 'Color',
    'is_4wd': '4WD',
    'date_posted': 'Fecha de posteo',
    'days_listed': 'Días publicado'
}, inplace=True)



# Transformaciones internas
cond_map = {
    'new': '**********',
    'like new': '********',
    'excellent': '******',
    'good': '****',
    'fair': '**',
    'salvage': '*'
}
df['Condición'] = df['Condición'].map(cond_map).fillna('-')

fuel_map = {
    'diesel': 'Diesel',
    'electric': 'Eléctrico',
    'gas': 'Gasolina',
    'hybrid': 'Híbrido',
    'other': 'Otro'
}
df['Combustible'] = df['Combustible'].map(fuel_map).fillna('Otro')

trans_map = {
    'automatic': 'Automático',
    'manual': 'Manual',
    'other': 'Otro'
}
df['Transmisión'] = df['Transmisión'].map(trans_map).fillna('Otro')

df['Tipo'] = df['Tipo'].str.title().fillna('Otro')
df['Tipo'] = df['Tipo'].replace({
    'Mini-Van': 'Mini-Van',
    'Suv': 'SUV',
    'Offroad': 'Off-Road',
    'Truck' : 'Camioneta',
    'Bus' : 'Autobus'
})

color_map = {
    'black': 'Negro',
    'white': 'Blanco',
    'silver': 'Plata',
    'blue': 'Azul',
    'red': 'Rojo',
    'grey': 'Gris',
    'green': 'Verde',
    'brown': 'Marrón',
    'orange': 'Naranja',
    'purple': 'Morado',
    'yellow': 'Amarillo',
    'custom': 'Personalizado',
    'other': 'Otro'
}
df['Color'] = df['Color'].map(color_map).fillna('Otro')
df['Color'] = df['Color'].apply(lambda x: x.capitalize())

# Fecha de posteo
df['Fecha de posteo'] = pd.to_datetime(df['Fecha de posteo'], errors='coerce')

# Días publicado
df['Días publicado'] = df['Días publicado'].fillna('-')

#Reordenamiento de las columnas
columnas_ordenadas=[
    'Precio', 'Marca', 'Modelo', 'Año del modelo', 
    'Condición', 'Cilindrada', 'Combustible', 'Kilometraje', 
    'Transmisión', 'Tipo', '4WD', 'Color', 
    'Fecha de posteo', 'Días publicado'
]
df = df[columnas_ordenadas]

# Reemplazar precios irreales (Precio == 1)
df_corregido = df.copy()

for idx in df_corregido[df_corregido['Precio'] == 1].index:
    marca = df_corregido.at[idx, 'Marca']
    modelo = df_corregido.at[idx, 'Modelo']
    tipo = df_corregido.at[idx, 'Tipo']

    condiciones = [
        (marca, modelo, tipo),
        (marca, modelo, None),
        (marca, None, tipo),
        (None, modelo, tipo)
    ]

    precio_median = None

    for cond in condiciones:
        m, mo, t = cond
        subset = df_corregido[df_corregido['Precio'] > 1]
        if m:
            subset = subset[subset['Marca'] == m]
        if mo:
            subset = subset[subset['Modelo'] == mo]
        if t:
            subset = subset[subset['Tipo'] == t]

        if not subset.empty:
            precio_median = subset['Precio'].median()
            break

    if precio_median is None:
        precio_median = df_corregido[df_corregido['Precio'] > 1]['Precio'].median()

    df_corregido.at[idx, 'Precio'] = int(precio_median)

df = df_corregido

# Guardar
print(f"Guardando archivo limpio en: {archivo_salida}")
df.to_csv(archivo_salida, index=False)

/tmp/ipykernel_34946/123946249.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['model'].fillna('Otro', inplace=True)
/tmp/ipykernel_34946/123946249.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

Guardando archivo limpio en: ../vehicles_us_limpio.csv
